# Gym Ignition Example

## Installation

In [0]:
# Disable stdout (use output.show() to print it)
%%capture --no-stderr output

# INSTALL IGNITION LIBRARIES
!\
    apt-get -qq update &&\
    echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-stable `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-stable.list &&\
    wget http://packages.osrfoundation.org/gazebo.key -O - | apt-key add - &&\
    apt update &&\
    apt install -y ignition-citadel

In [0]:
%%capture --no-stderr output
# INSTALL GYM IGNITION
!pip3 install gym-ignition

## CartPole: Random Policy

In [3]:
%%capture --no-stderr --no-stdout output

import gym
import gym_ignition
from gym_ignition.utils import logger

# Set gym verbosity
logger.set_level(gym.logger.INFO)
#logger.set_level(gym.logger.DEBUG)

logger.info("Initializing the environment")

# Create the environment
# env = gym.make("CartPole-v1")
env = gym.make("CartPoleDiscrete-Gazebo-v0")
# env = gym.make("CartPoleDiscrete-PyBullet-v0")

# Seed the environment
env.seed(42)

for epoch in range(10):
    # Reset the environment
    observation = env.reset()

    # Initialize returned values
    done = False
    total_reward = 0

    while not done:
        # Execute a random action
        action = env.action_space.sample()
        observation, reward, done, _ = env.step(action)

        # Render the environment
        # NOTE: You cannot render the environments in Colab
        # env.render('human')

        # Accumulate the reward
        total_reward += reward

        # Print the observation
        msg = ""
        for value in observation:
            msg += "\t%.6f" % value
        logger.debug(msg)

    logger.info("Total reward for episode #{}: {}".format(epoch, total_reward))

env.close()


INFO: Initializing the environment
INFO: Making new env: CartPoleDiscrete-Gazebo-v0


WARN: This runtime does not support setting the control mode


INFO: Total reward for episode #0: 384.0
INFO: Total reward for episode #1: 630.0
INFO: Total reward for episode #2: 413.0
INFO: Total reward for episode #3: 132.0
INFO: Total reward for episode #4: 189.0
INFO: Total reward for episode #5: 199.0
INFO: Total reward for episode #6: 226.0
INFO: Total reward for episode #7: 585.0
INFO: Total reward for episode #8: 534.0
INFO: Total reward for episode #9: 105.0
